<a href="https://colab.research.google.com/github/1972007/ml-c22-pc390/blob/main/C22_PC390.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Skin Disease Detection

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -q kaggle
! mkdir ~/.kaggle

In [ ]:
#Yang lain
from google.colab import files
json_kaggle = files.upload()
! cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle.json


In [4]:
#! kaggle datasets download shubhamgoel27/dermnet
# Ide dataset : https://www.kaggle.com/datasets/ammarabbasi1040/skin-diseases-dataset
! kaggle datasets download ismailpromus/skin-diseases-image-dataset
#! kaggle datasets download ammarabbasi1040/skin-diseases-dataset

 95% 404M/424M [00:04<00:00, 96.7MB/s]
100% 424M/424M [00:04<00:00, 93.1MB/s]


In [5]:
import zipfile
#local_zip = "/content/dermnet.zip"
#dest_zip = "/content/dermnet"
local_zip = "/content/skin-diseases-image-dataset.zip"
dest_zip = "/content/skin-diseases-image-dataset"
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(dest_zip)
zip_ref.close()

In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import random
import os
import shutil

In [7]:
img_dir_example = "/content/skin-diseases-image-dataset/IMG_CLASSES/1. Eczema 1677/0_0.jpg"
im = Image.open(img_dir_example)
img_size = im.size
print(img_size)

FileNotFoundError: ignored

In [ ]:
# filePath = "/content/dermnet"
# trainPath = "/content/dermnet/train/"
# testPath = "/content/dermnet/test/"
filePath = "/content/skin-diseases-image-dataset"
fileSrc = "/content/skin-diseases-image-dataset/IMG_CLASSES"
trainPath = os.path.join(filePath, "train")
testPath = os.path.join(filePath, "test")
folderr = sorted(os.listdir(fileSrc))
print(folderr)
split_rate = 0.9
try:
  os.mkdir(trainPath)
except:
  shutil.rmtree(trainPath)
  os.mkdir(trainPath)
try:
  os.mkdir(testPath)
except:
  shutil.rmtree(testPath)
  os.mkdir(testPath)
for dir_name in folderr :
  os.mkdir(os.path.join(trainPath, dir_name))
  os.mkdir(os.path.join(testPath, dir_name))

['1. Eczema 1677', '10. Warts Molluscum and other Viral Infections - 2103', '2. Melanoma 15.75k', '3. Atopic Dermatitis - 1.25k', '4. Basal Cell Carcinoma (BCC) 3323', '5. Melanocytic Nevi (NV) - 7970', '6. Benign Keratosis-like Lesions (BKL) 2624', '7. Psoriasis pictures Lichen Planus and related diseases - 2k', '8. Seborrheic Keratoses and other Benign Tumors - 1.8k', '9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k']


In [ ]:
split_rate = 0.9
for dir_name in folderr:
  dir_name_train = os.path.join(trainPath, dir_name) #Train folder with chosen subfolder
  dir_name_test = os.path.join(testPath, dir_name) #Test folder with chosen subfolder (the labels)
  src_folder = os.path.join(fileSrc, dir_name)
  src_files = os.listdir(os.path.join(fileSrc, dir_name)) #Src folder of chosen subfolder
  random.shuffle(src_files)
  split_num = int(len(src_files) * split_rate)
  print(dir_name_train,"\n",dir_name_test)
  for filee in src_files[:split_num]:
    src = os.path.join(src_folder, filee)
    dest = os.path.join(dir_name_train, filee)
    shutil.copyfile(src,dest)

  for filee in src_files[split_num:]:
    src = os.path.join(src_folder, filee)
    dest = os.path.join(dir_name_test, filee)
    shutil.copyfile(src,dest)

/content/skin-diseases-image-dataset/train/1. Eczema 1677 
 /content/skin-diseases-image-dataset/test/1. Eczema 1677
/content/skin-diseases-image-dataset/train/10. Warts Molluscum and other Viral Infections - 2103 
 /content/skin-diseases-image-dataset/test/10. Warts Molluscum and other Viral Infections - 2103
/content/skin-diseases-image-dataset/train/2. Melanoma 15.75k 
 /content/skin-diseases-image-dataset/test/2. Melanoma 15.75k
/content/skin-diseases-image-dataset/train/3. Atopic Dermatitis - 1.25k 
 /content/skin-diseases-image-dataset/test/3. Atopic Dermatitis - 1.25k
/content/skin-diseases-image-dataset/train/4. Basal Cell Carcinoma (BCC) 3323 
 /content/skin-diseases-image-dataset/test/4. Basal Cell Carcinoma (BCC) 3323
/content/skin-diseases-image-dataset/train/5. Melanocytic Nevi (NV) - 7970 
 /content/skin-diseases-image-dataset/test/5. Melanocytic Nevi (NV) - 7970
/content/skin-diseases-image-dataset/train/6. Benign Keratosis-like Lesions (BKL) 2624 
 /content/skin-disease

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255.0)
train_gen = train_datagen.flow_from_directory(directory=trainPath,
                                              batch_size=128,
                                              class_mode='categorical',
                                              target_size=img_size)

valid_datagen = ImageDataGenerator(rescale=1/255.0)
valid_gen = train_datagen.flow_from_directory(directory=testPath,
                                              batch_size=128,
                                              class_mode='categorical',
                                              target_size=img_size)

Found 24434 images belonging to 10 classes.
Found 2719 images belonging to 10 classes.


In [ ]:
output = len(folderr)
print(output)
activation = "softmax"
if output < 2 :
  activation = "sigmoid"
model = tf.keras.models.Sequential([ 
    tf.keras.layers.Conv2D(16, (3,3), activation="relu", input_shape=(294, 222,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(output,activation=activation)
])
 

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=['accuracy'])

10


In [ ]:
history = model.fit(train_gen,
                    epochs=15,
                    verbose=1,
                    validation_data=valid_gen)   

Epoch 1/15
191/191 [==============================] - 245s 1s/step - loss: 1.4533 - accuracy: 0.4713 - val_loss: 1.1586 - val_accuracy: 0.5656
Epoch 2/15
191/191 [==============================] - 245s 1s/step - loss: 1.0878 - accuracy: 0.5927 - val_loss: 1.0362 - val_accuracy: 0.6087
Epoch 3/15
191/191 [==============================] - 242s 1s/step - loss: 0.9752 - accuracy: 0.6334 - val_loss: 1.0177 - val_accuracy: 0.6204
Epoch 4/15
191/191 [==============================] - 242s 1s/step - loss: 0.8920 - accuracy: 0.6667 - val_loss: 0.9617 - val_accuracy: 0.6414
Epoch 5/15
191/191 [==============================] - 242s 1s/step - loss: 0.8225 - accuracy: 0.6940 - val_loss: 0.9558 - val_accuracy: 0.6410
Epoch 6/15
191/191 [==============================] - 243s 1s/step - loss: 0.7468 - accuracy: 0.7242 - val_loss: 0.9241 - val_accuracy: 0.6675
Epoch 7/15
191/191 [==============================] - 243s 1s/step - loss: 0.6528 - accuracy: 0.7599 - val_loss: 0.9564 - val_accuracy: 0.6646

In [ ]:
model.save("skin-diseases-image-dataset_model_v_1.h5")

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
model = tf.keras.models.load_model("test.h5")

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image


uploaded = files.upload()
for k, v in uploaded.items():
  img = image.load_img(k, target_size=img_size)
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)

  images = np.vstack([x])
  classes = model.predict(images,batch_size=10)
  print(k)
  print(classes)


['Acne and Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions', 'Atopic Dermatitis Photos', 'Bullous Disease Photos', 'Cellulitis Impetigo and other Bacterial Infections', 'Eczema Photos', 'Exanthems and Drug Eruptions', 'Hair Loss Photos Alopecia and other Hair Diseases', 'Herpes HPV and other STDs Photos', 'Light Diseases and Disorders of Pigmentation', 'Lupus and other Connective Tissue diseases', 'Melanoma Skin Cancer Nevi and Moles', 'Nail Fungus and other Nail Disease', 'Poison Ivy Photos and other Contact Dermatitis', 'Psoriasis pictures Lichen Planus and related diseases', 'Scabies Lyme Disease and other Infestations and Bites', 'Seborrheic Keratoses and other Benign Tumors', 'Systemic Disease', 'Tinea Ringworm Candidiasis and other Fungal Infections', 'Urticaria Hives', 'Vascular Tumors', 'Vasculitis Photos', 'Warts Molluscum and other Viral Infections']


Saving download.jpg to download.jpg
download.jpg
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
